# Note Book for Capstone Project - IBM Data Science Professional Certification

In [1]:
import pandas as pd
import numpy as np

# Libraries for web scraping
from bs4 import BeautifulSoup
import requests
import csv

**Reading the details from wikipedia to fetch postal codes for Canada using BeautifulSoup package**

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')

**Reading the web content into a csv file for reading directly into pandas dataframe**

In [3]:
data_table = soup.find('table', class_='wikitable sortable')
data_row = data_table.find_all('tr')

csv_file = open('canada_post_codes','w')
csv_writer = csv.writer(csv_file)

for line in data_row:
    p_ = line.text
    l_ = p_.split('\n')
    csv_writer.writerow([l_[1],l_[2],l_[3]])

csv_file.close()                         

In [4]:
df = pd.read_csv('canada_post_codes')

**Ignoring and removing all the Boroughs those are 'Not assigned'**

In [5]:
# Remove the rows with borough 'Not Assigned'
df = df[df.Borough!='Not assigned']
df = df.reset_index(drop=True)

In [6]:
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


**Replace all 'Not assigned' neighbourhoods with valid Boroughs in the dataframe**

In [8]:
## Replace all 'Not assigned' Neighborhoods with Borough

def neighborhood(row_):
    if row_['Neighbourhood'] == 'Not assigned': 
        x_ = row_['Borough']
    else:
        x_ = row_['Neighbourhood']
    return x_
    
for row in df.iterrows():
    df['Neighbourhood'] = df.apply(lambda row: neighborhood(row), axis=1)


**Build neighbourhoods with common Boroughs under same postal code**

In [9]:
# if the value count>1 you need to merge the neighbourhood
def merge_nh(row):
    cnt_ = df[df.Postcode == row[1][0]].shape[0]  
    if cnt_ > 1 :
       x_ = df[df.Postcode == row[1][0]]['Neighbourhood'].tolist()
       l_ = ','.join(x_)
    else:
       l_ = row[1][2]
    return l_

k=[]
for row in df.iterrows():
    k.append(merge_nh(row))

df['Neighbourhood'] = k

**Removing all duplicates**

In [10]:
df.drop_duplicates(keep='first',inplace=True)
df = df.reset_index(drop=True)

In [11]:
df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


**Shape of the dataframe**

In [12]:
# shape of the data frame
df.shape

(103, 3)

**Fetching the latitude and the longitude for each neighbourhood**

In [11]:
# !conda install -c conda-forge geocoder

In [12]:
import geocoder
# -------------------  IMPORTANT  -----------------------
# g = geocoder.google is not working. Always returns null

**Fetching the lat, lng details from csv file - http://cocl.us/Geospatial_data**

In [13]:
df_coords = pd.read_csv('http://cocl.us/Geospatial_data')
df_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
# Merging two dataframes on Postal Code
# Rename the Postal Code in coordinates data to Postcode
df_coords.rename(columns={'Postal Code':'Postcode'}, inplace=True)
df = pd.merge(df,df_coords,on='Postcode')

In [15]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
